In [ ]:
#title "Clinical analysis of a PAI cohort"
#author: Chris J Smith

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import re

#Upload data and clean data
raw_data = pd.read_csv("redcap_data_anon_trials.csv")

#assign ethnicity
def ethnicity_assigner(row):
    ethnicity = ""
    if row["ethnicity"] == 1:
        ethnicity = "Non-Finnish European"
    elif  row["ethnicity"] == 2:
        ethnicity = "Finnish"
    elif  row["ethnicity"] == 3:
        ethnicity = "African"
    elif  row["ethnicity"] == 4:
        ethnicity = "Latino"
    elif  row["ethnicity"] == 5:
        ethnicity = "South Asian"
    elif  row["ethnicity"] == 6:
        ethnicity = "Ashkenazi Jewish"
    elif  row["ethnicity"] == 7:
        ethnicity = "East Asian"
    elif  row["ethnicity"] == 8:
        ethnicity = "Other"
    else :
        ethnicity = "Unknown"
    return(ethnicity)

raw_data["ethnicity1"] = raw_data.apply(ethnicity_assigner, axis =1)

#assign consanguinity
def consanguinity_assigner(row):
    consanguinity = ""
    if row["consaguinity"] == 0:
        consanguinity = "Non-Consanguineous"
    elif row["consaguinity"] == 1:
        consanguinity= "Consanguineous"
    else :
        consanguinity="Unknown"
    return(consanguinity)

raw_data["consanguinity1"] = raw_data.apply(consanguinity_assigner, axis =1)

#assign hyperpigmentation
def hyperpigmentation_assigner(row):
    hyperpigmentation = ""
    if row["hyperpigmentation"] == 0:
        hyperpigmentation = "No Hyperpigmentation"
    elif row["hyperpigmentation"] == 1:
        hyperpigmentation= "Hyperpigmentation"
    else :
        hyperpigmentation="Unknown"
    return(hyperpigmentation)

raw_data["hyperpigmentation1"] = raw_data.apply(hyperpigmentation_assigner, axis =1)

#assign gender
def gender_assigner(row):
    gender = ""
    if row["gender___1"] == 1:
        gender = "Male"
    elif row["gender___2"] == 1:
        gender = "Female"
    else:
        gender = "Unknown"
    return(gender)

raw_data["gender"] = raw_data.apply(gender_assigner, axis =1)

#assign solved or unsolved
raw_data['solved'] = np.where(raw_data["mutation_solved"].isnull(),\
                            raw_data["mutation_solved2"],\
                            raw_data["mutation_solved"])

def solved_assigner(row):
    solved = ""
    if row["solved"]==0:
        solved = "Unsolved"
    elif row["solved"] ==1:
        solved = "Solved"
    else :
        solved = "Unkown"
    return(solved)

raw_data["solved1"] = raw_data.apply(solved_assigner, axis =1)

#ascertain gene defect
raw_data["gene1"] = np.where(raw_data["mutation_diagnosis"].isnull(),\
                            raw_data["mutation_diagnosis2"],\
                            raw_data["mutation_diagnosis"]).astype(str)

raw_data["gene"] = raw_data["gene1"].str.extract("(MC2R|MRAP|STAR|NROB1|AIRE|\
                                                |CYP11A1|NNT|AAAS|ABCD1|SGPL1| \
                                                |MCM4|TXNRD2|POMC|POLE|CYP21A2|\
                                                |POR|GPX1|CYP11B1|HSD3B2)", expand = False)

#select probands only 
raw_data = raw_data[raw_data["patient_or_family_member"]==1]

#drop none study id
data = raw_data.dropna(subset = ["study_id"])

#assign colours
colour_pal = {"AAAS":"#C0392B", "ABCD1":"#E74C3C","AIRE":"#a45ec0", "CYP11A1":"#8E44AD", 
           "CYP11B1":"#2980B9", "CYP21A2":"#3498DB", "GPX1":"#1ABC9C", "HSD3B2":"#837c19", 
           "MC2R":"#16A085", "MRAP":"#27AE60", "NNT":'#2ECC71', "NROB1":"#F1C40F", "POMC":"#F39C12", 
           "POR":"#E37E22","POLE":"#d3226c", "SGPL1": "#BDC37C", 
           "STAR":"#7F8C8D", "TXNRD2":"#34495E"}


In [ ]:
#Solved
solved = data[['study_id', 'gender', 'solved1', 'gene']]
solved_count = solved.value_counts('solved1')

#solved pie chart
solved_count.plot.pie(y='0')
plt.show()

In [ ]:
#Solved Gender
solved_gender = solved.groupby('solved1')['gender'].value_counts()

solved_gender.plot.pie(y='gender')
plt.show()

In [ ]:
#Genes pie
genes_count = pd.DataFrame(data['gene'].value_counts()).reset_index()
genes_count.rename(columns={"gene":"count", "index":"gene"} ,inplace=True)
plt.pie(genes_count["count"], labels= genes_count["gene"])
plt.show()

In [ ]:
#Genes Bar
sns.set_style("whitegrid")
sns.set_context("paper")
g = sns.catplot(x="gene", y="count", data=genes_count,
            kind="bar",
            palette=colour_pal        
            )
g.set(xlabel="Gene",
      ylabel="Number of Diagnosed Patients")
g.fig.suptitle("Cohort PAI Genes")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Genes with Genders

genes_genders = pd.DataFrame(data[["gene", "gender"]]).dropna(subset = "gene").reset_index(drop=True)
genes_genders = genes_genders.groupby(['gene','gender']).value_counts().unstack().fillna(0)
genes_genders["sum"] = (genes_genders["Female"] + genes_genders["Male"] + genes_genders["Unknown"])
genes_genders = genes_genders.sort_values(by = "sum", ascending= False)
genes_genders = genes_genders.drop('sum', axis = 1)
print(genes_genders)

# create stacked bar chart for df
genes_genders.plot(kind='bar', stacked=True, color=['blue', 'orange', 'green'])
 
# Add Title and Labels
plt.title('Genes with Genders')
plt.xlabel('Gene')
plt.ylabel('Number of diagnosed Patients')
plt.xticks(rotation=45)

plt.show()

In [ ]:
#cortisol boxplot

cortisol = data[["study_id", "gene", "cortisol_level3"]].dropna(subset = ["cortisol_level3", "gene"])\
           .sort_values(by = "gene")

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=cortisol, x="gene", y="cortisol_level3",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=cortisol, x = "gene", y = "cortisol_level3",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('Cortisol Levels', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('Cortisol (nmol/L)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 16)
plt.yticks( fontsize = 12)

plt.show()

In [ ]:
#acth boxplot

acth = data[["study_id", "gene", "acth_calc"]].dropna(subset = ["acth_calc", "gene"])\
           .sort_values(by = "gene")

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=acth, x="gene", y="acth_calc",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=acth, x = "gene", y = "acth_calc",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('ACTH Levels', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('ACTH (pmol/L)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 16)
plt.yticks( fontsize = 12)

plt.show()

In [ ]:
#acth:cortisol ratio

cort_acth = data[["study_id", "gene", "acth_calc", "cortisol_level3"]]\
                 .dropna(subset = ["gene", "acth_calc", "cortisol_level3"]).reset_index(drop = True)

cort_acth["ratio"] = cort_acth["acth_calc"]/cort_acth["cortisol_level3"]
cort_acth = cort_acth[cort_acth["ratio"]<5000].sort_values(by = "gene")

#print(cort_acth)

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=cort_acth, x="gene", y="ratio",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=cort_acth, x = "gene", y = "ratio",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('ACTH:Cortisol Levels', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('ACTH/Cortisol', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 12)
plt.yticks( fontsize = 16)

plt.show()

In [ ]:
#aldosterone boxplot

aldosterone = data[["study_id", "gene", "aldosterone_calc"]].dropna(subset = ["aldosterone_calc", "gene"])\
           .sort_values(by = "gene")

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=aldosterone, x="gene", y="aldosterone_calc",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=aldosterone, x = "gene", y = "aldosterone_calc",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('Aldosterone Levels', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('Aldosterone (pmol/L)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 12)
plt.yticks( fontsize = 16)

print(aldosterone.head(10))
plt.show()

In [ ]:
#dheas boxplot

dheas = data[["study_id", "gene", "dheas_calc"]].dropna(subset = ["dheas_calc", "gene"])\
           .sort_values(by = "gene")

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=dheas, x="gene", y="dheas_calc",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=dheas, x = "gene", y = "dheas_calc",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('DHEAS Levels', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('DHEAS (umol/L)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 12)
plt.yticks( fontsize = 16)

print(dheas.head(10))
plt.show()

In [ ]:
#renin boxplot

renin = data[["study_id", "gene", "renin_calc"]].dropna(subset = ["renin_calc", "gene"])\
           .sort_values(by = "gene")

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=renin, x="gene", y="renin_calc",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=renin, x = "gene", y = "renin_calc",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('Renin Levels', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('Renin (pmol/L)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 12)
plt.yticks( fontsize = 16)

print(renin.head(10))
plt.show()

In [ ]:
#Birth weight

bw = data[["study_id", "gene", "birth_weight"]].dropna(subset = ["gene", "birth_weight"]).sort_values(by = "gene")
bw = bw.replace(',','.', regex = True)
bw_ex = ["Normal", " -", "average", "NORMAL", "Normal "]
bw = bw[~bw.birth_weight.isin(bw_ex)]
bw["weight1"] = bw.birth_weight.str.extract('(\d+\.?\d*)').astype(float)
bw["weight"] = np.where(bw["weight1"] > 1000, \
               bw["weight1"] / 1000, \
               bw["weight1"]).astype(float)

print(bw)

plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=bw, x="gene", y="weight",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=bw, x = "gene", y = "weight",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('Birth Weight', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('Birth Weight (kg)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 12)
plt.yticks( fontsize = 16)


plt.show()

In [ ]:
#Age

age = data[["study_id", "gene","age_of_onset"]].dropna(subset = ["gene", "age_of_onset"])\
      .reset_index(drop=True).sort_values(by = "gene")
age["number"] = age.age_of_onset.str.extract('(\d+)').astype(float)

def age_assigner(row):
  years =  ""
  if re.search('birth|Birth|neonatal|Neonatal|early_life', row["age_of_onset"]):
      years = 0.1
  elif re.search('years|year|YEARS', row["age_of_onset"]):
    years = row["number"]
  elif re.search('month|months|m', row["age_of_onset"]):
    years =  row["number"] / 12
  elif re.search('weeks', row["age_of_onset"]):
    years = row["number"] / 52
  elif re.search('days|day', row["age_of_onset"]):
    years =  row["number"] / 365
  elif re.search('hours', row["age_of_onset"]):
    years =  row["number"] / (365 * 24)
  else:
    years = row["number"]
  return (years)

age["age_years"] = age.apply(age_assigner, axis = 1)
age = age[age["age_years"] < 20]
#print(age)


plt.figure(figsize=(12,8))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.boxplot(data=age, x="gene", y="age_years",
            palette=colour_pal,
            whis=[5,95],
            flierprops={"marker":"o", 'alpha':0.5},
            boxprops={'alpha': 0.4})
sns.swarmplot(data=age, x = "gene", y = "age_years",
              dodge=True,
              color = 'black',
              alpha = 0.5)
plt.title('Age of Onset', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('Age (years)', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 12)
plt.yticks( fontsize = 16)


plt.show()

In [ ]:
#Genes and ethnicity

ethnicity = data[["study_id", "gene", "ethnicity1"]].dropna(subset = "gene").reset_index(drop = True)
ethnicity = ethnicity[ethnicity["ethnicity1"] != "Unknown"]
ethnicity = ethnicity.groupby("gene")["ethnicity1"].value_counts().unstack().fillna(0)

ethnicity.plot(kind='bar', stacked=True)
 
# Add Title and Labels
plt.title('Genes with Ethnicity')
plt.xlabel('Gene')
plt.ylabel('Number of diagnosed Patients')
plt.xticks(rotation=45)

plt.show()


print(ethnicity)

In [ ]:
#Genes and ethnicity

ethnicity = data[["study_id", "gene", "ethnicity1"]].dropna(subset = "gene").reset_index(drop = True)
ethnicity = ethnicity[ethnicity["ethnicity1"] != "Unknown"]
ethnicity = ethnicity.groupby("ethnicity1")["gene"].value_counts().unstack().fillna(0)

plt.figure(figsize=(8,6))
ethnicity.plot(kind='bar', stacked=True)
 
# Add Title and Labels
plt.title('Genes with Ethnicity')
plt.xlabel('Gene')
plt.ylabel('Number of diagnosed Patients')
plt.xticks(rotation=45)

plt.show()
plt.savefig("test.pdf")
print(ethnicity)

In [ ]:
#Country of Referral
country = pd.DataFrame(data["country"].value_counts()).reset_index()
country.rename(columns={"country":"count", "index":"country"} ,inplace=True)

#fig, ax = plt.subplots()
plt.figure(figsize=(20,14))
sns.set_style("whitegrid")
sns.set_context("paper")
sns.barplot(x="country", y="count", data=country)
plt.title('Countries of Referral', fontdict={"fontsize":20})
plt.xlabel('Gene', fontdict={"fontsize":20})
plt.ylabel('Number of diagnosed Patients', fontdict={"fontsize":20})
plt.xticks(rotation=45, fontsize = 20)
plt.yticks( fontsize = 20)
plt.savefig("test.pdf", dpi =300)

plt.show()

#print(country)

In [ ]:
#map
import geopandas as gpd
import matplotlib.patches as mpatches

#Download world map
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

#Produce a list of countries and add countris that are in a different format
country_list = country["country"].tolist()

countries_to_add = ["United States of America",
                    "United Kingdom",
                    "United Arab Emirates"]

country_list = country_list + countries_to_add

#Select countries to colour
map_countries = world[world['name'].isin(country_list)]

df = world["name"]

# initialize an empty figure and add an axis
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

# plot a basic map of the world
world.plot(
    ax=ax,
    color="lightgray",
    edgecolor="black",
    alpha=0.5
)
#Colour counties in list
map_countries.plot(
    ax=ax,
    color="red",
    edgecolor="black",
    alpha=0.5
)
# turn off axis ticks
ax.set_xticks([])
ax.set_yticks([])

# set the plot title
plt.title("Countries of our International Cohort", fontdict={"fontsize":20})
plt.show()
plt.savefig("map of cohort.pdf", dpi =300)

